In [3]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Create a linear programming problem
model = LpProblem("Coal_Shipping_Problem", LpMinimize)

# Decision variables
mines = [1, 2, 3]
plants = [1, 2, 3, 4]

# Amount of coal to be shipped from each mine to each plant
x = LpVariable.dicts("coal_shipped", ((i, j) for i in mines for j in plants), lowBound=0, cat='Integer')

# Objective function: Minimize total cost
model += lpSum(62 * x[i, j] for i in mines for j in plants) + \
         lpSum(7 * x[i, j] for i in mines for j in plants if j == 1) + \
         lpSum(9 * x[i, j] for i in mines for j in plants if j == 2) + \
         lpSum(14 * x[i, j] for i in mines for j in plants if j == 3)

# Constraints
# Mine capacity constraints
mine_capacity = {1: 220, 2: 190, 3: 280}
for i in mines:
    model += lpSum(x[i, j] for j in plants) <= mine_capacity[i]

# Plant demand constraints
plant_demand = {1: 110, 2: 160, 3: 90, 4: 180}
for j in plants:
    model += lpSum(x[i, j] for i in mines) >= plant_demand[j]

# Ash and sulfur content constraints
for j in plants:
    model += 0.09 * x[1, j] + 0.05 * x[2, j] + 0.04 * x[3, j] <= 0.06 * lpSum(x[i, j] for i in mines)
    model += 0.06 * x[1, j] + 0.04 * x[2, j] + 0.03 * x[3, j] <= 0.05 * lpSum(x[i, j] for i in mines)

# Solve the problem
model.solve()

# Print the results
print(f"Status: {model.status}")
print(f"Total Cost: ${model.objective.value()}")

for i in mines:
    for j in plants:
        print(f"Amount of coal shipped from Mine {i} to Plant {j}: {x[i, j].value()} tons")


Status: 1
Total Cost: $36950.0
Amount of coal shipped from Mine 1 to Plant 1: 33.0 tons
Amount of coal shipped from Mine 1 to Plant 2: 64.0 tons
Amount of coal shipped from Mine 1 to Plant 3: 36.0 tons
Amount of coal shipped from Mine 1 to Plant 4: 72.0 tons
Amount of coal shipped from Mine 2 to Plant 1: 55.0 tons
Amount of coal shipped from Mine 2 to Plant 2: 0.0 tons
Amount of coal shipped from Mine 2 to Plant 3: 0.0 tons
Amount of coal shipped from Mine 2 to Plant 4: 0.0 tons
Amount of coal shipped from Mine 3 to Plant 1: 22.0 tons
Amount of coal shipped from Mine 3 to Plant 2: 96.0 tons
Amount of coal shipped from Mine 3 to Plant 3: 54.0 tons
Amount of coal shipped from Mine 3 to Plant 4: 108.0 tons
